In [ ]:
!pip install ultralytics==8.0.196
!pip install roboflow

In [ ]:
from ultralytics import YOLO
from roboflow import Roboflow
import os
from IPython.display import display, Image
import glob

In [ ]:
# 체크
!yolo mode=checks

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("project4mealmakerbox-wiokd").project("foodproject-8rpvr")
version = project.version(7)
dataset = version.download("yolov8")

In [ ]:
# 훈련

!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=100, imgsz=640 save=True

WARNING ⚠️ 'epochs=100,' does not require trailing comma ',', updating to 'epochs=100'.
100% 6.23M/6.23M [00:00<00:00, 21.1MB/s]
New https://pypi.org/project/ultralytics/8.2.68 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/FoodProject-7/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, sa

In [ ]:
display(Image(filename=f'/content/runs/detect/train/confusion_matrix.png', width=600))
display(Image(filename=f'/content/runs/detect/train/results.png', width=600))
display(Image(filename=f'/content/runs/detect/val/confusion_matrix_normalized.png', width=600))

In [ ]:
# 검증
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
display(Image(filename=f'/content/runs/detect/val/val_batch0_pred.jpg', width=600))

In [ ]:
# 예측 실행 및 결과 시각화
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.5 source=/content/test_img

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3014228 parameters, 0 gradients, 8.1 GFLOPs

image 1/7 /content/test_img/test1.jpeg: 480x640 1 egg, 1 garlic, 1 green onion, 1 onion, 1 pepper, 1 spam, 1 tuna can, 61.4ms
image 2/7 /content/test_img/test2.jpeg: 640x480 1 egg, 1 green onion, 1 onion, 1 pepper, 1 sweet potato, 1 tofu, 1 tuna can, 60.5ms
image 3/7 /content/test_img/test3.png: 256x640 1 beef, 1 pork, 61.4ms
image 4/7 /content/test_img/test4.png: 416x640 1 Bean sprouts, 1 mung bean sprout, 59.4ms
image 5/7 /content/test_img/test5.png: 352x640 1 Bean sprouts, 1 mung bean sprout, 59.0ms
image 6/7 /content/test_img/test6.jpg: 640x480 1 Enoki Mushroom, 1 egg, 1 pepper, 1 tofu, 1 tuna can, 6.8ms
image 7/7 /content/test_img/test7.jpeg: 640x480 1 cheese, 1 egg, 1 kimchi, 1 ramen, 6.8ms
Speed: 2.8ms preprocess, 45.1ms inference, 83.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect

In [ ]:
# 예측 이미지 표시
image_pattern = '/content/runs/detect/predict/*'

for image_path in glob.glob(image_pattern):
  if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):
    display(Image(filename=image_path, height=600))
    print('\n')